# IBM Data Science Professional Certificate

##### Applied Data Science Week 3 Assignment: Segmenting and Clustering Neighborhoods in Toronto
##### Coursera: https://www.coursera.org/learn/applied-data-science-capstone

##### Preprocessing: Prepare for arranged dataframe from section 1 & 2

In [259]:
from bs4 import BeautifulSoup
import requests
import lxml
import csv
import pandas as pd
import numpy as np
import geopy

In [255]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
#table = soup.find('table', class_ ='wikitable sortable')
header = table.find_all('th')
hd = []
for h in header:
    head = h.text
    hd.append(head)
hd2 = [x.strip('\n') for x in hd]
hd2[0] = 'PostalCode'

rows = table.find_all('tr')
rw = []
for row in rows:
    cell = row.find_all('td')
    row = [i.text for i in cell]
    row2 = [x.strip('\n') for x in row]
    rw.append(row2)
rw2 = [x for x in rw if x != []]

df = pd.DataFrame(rw2)
df.columns = [hd2]

df.to_csv('df.csv', index=False)
df1 = pd.read_csv('df.csv')
df2 = df1.dropna(axis=0)
df2.reset_index(drop=True)
df3 = df2
df3.Neighborhood = df3.Neighborhood.str.replace('/',',') 

geo_data = pd.read_csv('http://cocl.us/Geospatial_data')
geo = pd.DataFrame(geo_data)
geo = geo.rename(columns={'Postal Code':'PostalCode'})

df4 = df3.merge(geo,how='outer')
df4.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [256]:
df4.shape

(103, 5)

### Section 3: Explore and cluster the neighborhoods in Toronto

In [260]:
import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


##### 3.1 Get location infomation from Toronto and create a map from the neighborhoods in Toronto

Use geopy module to get the geographic coordinates of Toronto

In [261]:
address = 'Toronto, Ontario'

geolocator = geopy.Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.6534817, -79.3839347.


Input the relavant account information for Foursquare to get the credentails

In [237]:
CLIENT_ID = 'AAGM420Z2D0G102XKRKQC2J5TWW3PNJNOIFNTTYX1Z1ZHBLS' # your Foursquare ID
CLIENT_SECRET = 'JQZHTIGGI14MFDOWR4UFTLNMFTLYTBHOTCMARIZPYZFFQPU5' # your Foursquare Secret
VERSION = '20200505' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AAGM420Z2D0G102XKRKQC2J5TWW3PNJNOIFNTTYX1Z1ZHBLS
CLIENT_SECRET:JQZHTIGGI14MFDOWR4UFTLNMFTLYTBHOTCMARIZPYZFFQPU5


Use the function built from lab to get information of venues nearby each neighborhood

In [315]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 5
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'NBH Latitude', 
                  'NBH Longitude', 
                  'Venue', 
                  'VN Latitude', 
                  'VN Longitude', 
                  'Category']
    
    return(nearby_venues)

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Filter the neighborhoods in boroughs that only contain the word 'Toronto'

In [317]:
toronto_data = df4.set_index('Borough').filter(regex=".*Toronto",axis=0)
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District , Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Rosed

Create the map of above filtered neighborhoods with folium  module
(I think the map could not be displayed in Github, so you can find it here: https://drive.google.com/file/d/1qamwJ2RS9dIi1z5-Qnt3EfHOFAOI7OXW/view )

In [263]:
map_to = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

Estimate the venues shape in Toronto and calculate the number of unique venues categories

In [201]:
print(toronto_venues.shape)
toronto_venues.head()
toronto_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Category'].unique())))

(1526, 7)
There are 241 uniques categories.


##### 3.2 Explore the top 10 most common venues nearby each neighborhood

Data processing: one hot encoding

In [202]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group the venues by neighborhoods and calculate the shape

In [239]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(66, 241)

Estimate the frequency of top 5 venues by neighborhoods

In [204]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor , Wilson Heights , Downsview North----
         venue  freq
0         Bank  0.11
1  Coffee Shop  0.11
2        Diner  0.05
3  Gas Station  0.05
4  Bridal Shop  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3   Chinese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park , Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.08
2          Restaurant  0.08
3      Sandwich Place  0.08
4    Sushi Restaurant  0.08


----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.06
2            Beer Bar  0.04
3  Seafood Restaurant  0.04
4                Café  0.04


----Brockton , Parkdale Village , Exhibition Place----
            venue  freq
0            Café  0.13
1     Coffee Shop  0.09
2       Nightclub  0.09
3  Breakfast Spot  0.09
4         Stadium  0.04


----Busine

Expand the number to top 10 venues of neighborhoods and create the dataframe

In [269]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{} th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
0,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Shopping Mall,Pizza Place,Diner,Restaurant,Bridal Shop,Ice Cream Shop,Deli / Bodega,Supermarket
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Bank,Café,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
2,"Bedford Park , Lawrence Manor East",Sushi Restaurant,Restaurant,Italian Restaurant,Sandwich Place,Coffee Shop,Butcher,Pharmacy,Indian Restaurant,Café,Liquor Store
3,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Seafood Restaurant,Cheese Shop,Café,Restaurant,Japanese Restaurant,Hotel
4,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Nightclub,Breakfast Spot,Stadium,Bakery,Burrito Place,Intersection,Italian Restaurant,Restaurant


Set up parameters for k-means clustering

In [270]:
# set number of clusters
k = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=3).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2],
      dtype=int32)

Insert the k-means cluster labels into the dataframe

In [271]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Label', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Neighborhood,Latitude,Longitude,Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
Borough,,,,,,,,,,,,,,,
Downtown Toronto,M5A,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Café,Theater,Mexican Restaurant,Hotel,Ice Cream Shop
Downtown Toronto,M7A,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Creperie,Beer Bar,Sandwich Place,Burger Joint,Burrito Place,Café
Downtown Toronto,M5B,"Garden District , Ryerson",43.657162,-79.378937,0,Café,Middle Eastern Restaurant,Clothing Store,Restaurant,Bookstore,Ramen Restaurant,Theater,Cosmetics Shop,Tea Room,Coffee Shop
Downtown Toronto,M5C,St. James Town,43.651494,-79.375418,0,Café,Gastropub,Coffee Shop,Seafood Restaurant,Hotel,Cosmetics Shop,Creperie,American Restaurant,Restaurant,Farmers Market
East Toronto,M4E,The Beaches,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Curling Ice


Create clustering map

(As already stated, you can find the map here: https://drive.google.com/file/d/1JbFSlJlcSEvowZv_9DRlDPXpTLqB77fU/view )

In [272]:
# create map
latitude = 43.654260
longitude = -79.520999

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Retrieve data from each clustered neighborhood in details by sorted labels

##### Cluster 1

In [273]:
toronto_merged.loc[toronto_merged['Label'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
Borough,,,,,,,,,,,
Downtown Toronto,"Regent Park , Harbourfront",Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Café,Theater,Mexican Restaurant,Hotel,Ice Cream Shop
Downtown Toronto,"Queen's Park , Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Creperie,Beer Bar,Sandwich Place,Burger Joint,Burrito Place,Café
Downtown Toronto,"Garden District , Ryerson",Café,Middle Eastern Restaurant,Clothing Store,Restaurant,Bookstore,Ramen Restaurant,Theater,Cosmetics Shop,Tea Room,Coffee Shop
Downtown Toronto,St. James Town,Café,Gastropub,Coffee Shop,Seafood Restaurant,Hotel,Cosmetics Shop,Creperie,American Restaurant,Restaurant,Farmers Market
East Toronto,The Beaches,Trail,Pub,Health Food Store,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Curling Ice
Downtown Toronto,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Seafood Restaurant,Cheese Shop,Café,Restaurant,Japanese Restaurant,Hotel
Downtown Toronto,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Ice Cream Shop,Burger Joint,Yoga Studio,Modern European Restaurant,Salad Place
Downtown Toronto,Christie,Grocery Store,Café,Park,Italian Restaurant,Restaurant,Athletics & Sports,Candy Store,Diner,Baby Store,Nightclub
Downtown Toronto,"Richmond , Adelaide , King",Coffee Shop,Café,Steakhouse,Concert Hall,Restaurant,Pizza Place,American Restaurant,Seafood Restaurant,Opera House,Colombian Restaurant


##### Cluster 2

In [669]:
toronto_merged.loc[toronto_merged['Label'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
Borough,,,,,,,,,,,
Central Toronto,Roselawn,Garden,Women's Store,Deli / Bodega,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


##### Cluster 3

In [670]:
toronto_merged.loc[toronto_merged['Label'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
Borough,,,,,,,,,,,
Central Toronto,"Moore Park , Summerhill East",Park,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


##### Cluster 4

In [671]:
toronto_merged.loc[toronto_merged['Label'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
Borough,,,,,,,,,,,
Central Toronto,Forest Hill North & West,Park,Jewelry Store,Trail,Bus Line,Sushi Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
Downtown Toronto,Rosedale,Park,Playground,Trail,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


##### Cluster 5

In [672]:
toronto_merged.loc[toronto_merged['Label'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4 th Most Common Venue,5 th Most Common Venue,6 th Most Common Venue,7 th Most Common Venue,8 th Most Common Venue,9 th Most Common Venue,10 th Most Common Venue
Borough,,,,,,,,,,,
Central Toronto,Lawrence Park,Swim School,Park,Construction & Landscaping,Bus Line,Women's Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


##### Some findings and explanations of the clustering process:

As we can see, the neighborhoods are grouped into 5 clusters.

Group 1 : the majority, their most common venues are mostly daily life facilities or gastronomies(e.g. Cafés, restaurants, shops...)  
Group 2 : contains only 1 neighborhood 'Roselawn', where the most common venue is garden  
Group 3 : contains only 1 neighborhood 'Moore Park , Summerhill East', where the most popular venue is park  
Group 4 : contains 2 neighborhoods, whose most common venues are also parks (just the same as group 2)  
Group 5 : contains only 1 neighborhood 'Lawrence Park', where the most common venues is swim pool  

I personally think group 3 and group 4 can be sorted into the same group, as they have very similar venues structures.  
Moreover, several neighborhoods are not correctly categorized, for example neighborhood 'Forest Hill North & West' (in Borough Central Toronto) from group 1, whose most common venue is park, which should be grouped into group 3 or 4.